# 나이 정제 및 품질 등급 부여

In [6]:
import sys
import pandas as pd

sys.path.append("../src")
from clean_age import clean_age


sample_ages = [
    '18', '20세', 'twenty-one', '0x15', 'n/a', '-', '나이:30', '', 'seventeen'
]

df_test = pd.DataFrame({'원본나이': sample_ages})

df_test[['정제된_나이', '나이_등급']] = df_test['원본나이'].apply(lambda x: pd.Series(clean_age(x)))

df_test

,원본나이,정제된_나이,나이_등급
0,18,18.0,A
1,20세,20.0,A
2,twenty-one,21.0,B
3,0x15,21.0,B
4,n/a,NaN,C
5,-,NaN,C
6,나이:30,30.0,A
7,,NaN,C
8,seventeen,17.0,B


# 라이브러리 및 모듈 로딩

In [15]:
import pandas as pd
from pathlib import Path

from clean_age import clean_age
from clean_score import clean_score

import os

RAW_PATH = Path("../raw/messy_student_scores_500.csv")
SAVE_PATH = Path("../data/processed/cleaned_student_scores_csv")

os.makedirs(SAVE_PATH.parent, exist_ok=True)



# 원본 데이터 로딩

In [16]:
df = pd.read_csv(RAW_PATH)
print(df.shape)
df.head()

(500, 6)


,이름,나이,성별,점수,시험일,비고
0,NaN,0x1d,남,NaN,19.09.1979,NaN
1,서정남,twenty,male,59점,NaN,NaN
2,김영자,NaN,male,ninety-five,NaN,.
3,NaN,10세,male,none,10-06-1980,retake
4,김재호,NaN,m,90점,1979년 05월 16일,없음


# 정제 적용: 점수 + 나이 정제 함수 동시 적용

In [12]:
df[['정제된_점수', 'score_quality']] = df['점수'].apply(lambda x: pd.Series(clean_score(x))) 
df[['정제된_나이', 'age_quality']] = df['나이'].apply(lambda x: pd.Series(clean_age(x)))

# 정제 결과 확인 및 결측치 처리

In [13]:
df_cleaned = df.copy()

df_cleaned = df_cleaned.dropna(subset=['정제된_점수', '정제된_나이'])

print(f"정제 후 남은 행 수: {len(df_cleaned)}")
df_cleaned[['이름', '나이', '점수', '정제된_나이', 'age_quality', 'score_quality']].head(10)


정제 후 남은 행 수: 106


,이름,나이,점수,정제된_나이,age_quality,score_quality
1,서정남,twenty,59점,20.0,B,A
11,NaN,15,0b1010011,15.0,A,B
13,한경희,seventeen,96,17.0,B,A
19,김영미,seventeen,93점,17.0,B,A
20,윤윤서,18,64,18.0,A,A
21,홍지영,21세,52,21.0,A,A
28,NaN,twenty,52점,20.0,B,A
29,김명숙,19세,0b1100011,19.0,A,B
31,김성훈,seventeen,0b111110,17.0,B,B
44,하미경,twenty,55점,20.0,B,A


# 정제된 파일 저장

In [17]:
df_cleaned.to_csv(SAVE_PATH, index=False)
print(f"정제된 데이터 저장 완료: {SAVE_PATH}")

정제된 데이터 저장 완료: ..\data\processed\cleaned_student_scores_csv
